## Search Engine with Tools and Agents

In [ ]:
#tools creation
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper


In [ ]:
#used inbuilt TOOL for WIKIPEDIA
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)

wiki.name

'wikipedia'

In [4]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [5]:
tools = [wiki,arxiv]

In [14]:
# Create Custom TOOL
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [15]:
os.environ['HF_TOKEN'] = os.getenv("HF_TOKEN")
embeddings = HuggingFaceEmbeddings(model_name="all-miniLM-L6-v2")

In [16]:
loader = WebBaseLoader("https://docs.langchain.com/oss/python/langchain/overview")
data = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(data)


vectordb = FAISS.from_documents(documents, embedding=embeddings)

retriever = vectordb.as_retriever()

retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x127e23250>, search_kwargs={})

In [17]:
#Create a TOOL from our retriever

from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "LangSmithDocs",
    "Search the LangSmith documentation to find relevant information")

retriever_tool.name

'LangSmithDocs'

In [18]:
tools = [arxiv, wiki, retriever_tool]
tools

[ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/prathmesh/LangChain/.venv/lib/python3.13/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='LangSmithDocs', description='Search the LangSmith documentation to find relevant information', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x117cc4040>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at

## Run the above CUSTOM TOOL with Agent and LLM

In [19]:
# TOOLS and LLM--> agent executor
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq
groq_api_key = os.getenv("GROQ_API_KEY")

llm = ChatGroq(api_key=groq_api_key, model="llama-3.1-8b-instant")

In [20]:
## PROMPT TEMPLATE for Agent
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [21]:
##Agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(
    llm,
    tools,
    prompt
)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [23]:
## Agent Executer

from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [25]:
agent_executor.invoke({"input":"What is Langchain?"})



> Entering new AgentExecutor chain...

Invoking: `LangSmithDocs` with `{'query': 'Langchain'}`


See the Installation instructions and Quickstart guide to get started building your own agents and applications with LangChain.
​ Core benefits

LangChain overview - Docs by LangChainSkip to main contentDocs by LangChain home pageLangChain + LangGraphSearch...⌘KSupportGitHubTry LangSmithTry LangSmithSearch...NavigationLangChain overviewLangChainLangGraphDeep AgentsIntegrationsLearnReferenceContributePythonOverviewGet startedInstallQuickstartChangelogPhilosophyCore componentsAgentsModelsMessagesToolsShort-term memoryStreamingStructured outputMiddlewareOverviewBuilt-in middlewareCustom middlewareAdvanced usageGuardrailsRuntimeContext engineeringModel Context Protocol (MCP)Human-in-the-loopMulti-agentRetrievalLong-term memoryAgent developmentLangSmith StudioTestAgent Chat UIDeploy with LangSmithDeploymentObservabilityOn this page Create an agent Core benefitsLangChain overviewCopy pageLangCh

{'input': 'What is Langchain?',
 'output': "It seems like LangChain and LangGraph are part of the same ecosystem, and the choice between them would depend on the specific needs of your project. If you're looking for a more straightforward and easy-to-use solution, LangChain might be the better choice. However, if you need more advanced features, customization, or deterministic workflows, LangGraph might be the better fit.\n\nIf you're still unsure, you could consider the following factors:\n\n1. Complexity: If your project requires a high degree of customization or complex workflows, LangGraph might be more suitable.\n2. Ease of use: If you prioritize ease of use and a simple, intuitive API, LangChain might be the better choice.\n3. Scalability: If you anticipate your project will require large-scale deployment or high-performance processing, LangGraph's built-in support for durable execution and human-in-the-loop might be beneficial.\n4. Integration: If you need to integrate with othe

In [26]:
agent_executor.invoke({"input":"What is Tool and Agent in Langchain?"})



> Entering new AgentExecutor chain...

Invoking: `LangSmithDocs` with `{'query': 'Tool and Agent in Langchain'}`


We recommend you use LangChain if you want to quickly build agents and autonomous applications. Use LangGraph, our low-level agent orchestration framework and runtime, when you have more advanced needs that require a combination of deterministic and agentic workflows, heavy customization, and carefully controlled latency.
LangChain agents are built on top of LangGraph in order to provide durable execution, streaming, human-in-the-loop, persistence, and more. You do not need to know LangGraph for basic LangChain agent usage.
​ Create an agent
Copy# pip install -qU langchain "langchain[anthropic]"
from langchain.agents import create_agent

​ Core benefits
Standard model interfaceDifferent providers have unique APIs for interacting with models, including the format of responses. LangChain standardizes how you interact with models so that you can seamlessly swap providers an

{'input': 'What is Tool and Agent in Langchain?',
 'output': 'From the documentation, it seems that in the context of LangChain, "Agent" and "Tool" are not explicitly defined. However, based on the provided information, we can infer that:\n\n*   "Agent" refers to the pre-built agent architecture provided by LangChain, which allows users to build autonomous applications and adapt to the ecosystem.\n*   "Tool" is not explicitly defined, but it is mentioned as one of the components that LangChain integrates with, along with models. This suggests that a "Tool" is likely a specific software or library that can be used in conjunction with LangChain\'s agent architecture.\n\nIf you\'re looking for more information on the specific definitions of "Agent" and "Tool" in the context of LangChain, I recommend checking the official documentation or reaching out to the LangChain community for clarification.'}

In [28]:
agent_executor.invoke({"input":"What is this research paper '1706.03762' about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on cIt looks like the paper '1706.03762' is about the Transformer model, specifically the original paper that introduced it by Vaswani et al. in 2017.

> Finished chain.


{'input': "What is this research paper '1706.03762' about?",
 'output': "It looks like the paper '1706.03762' is about the Transformer model, specifically the original paper that introduced it by Vaswani et al. in 2017."}